# Standard Algorithm for Nearest Neighbours

In [1]:
import openml
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer

In [2]:
import pandas as pd
import numpy as np
import scipy.stats as ss
import scikit_posthocs as sp # Provavelmente vais ter de correr !pip install scikit-posthocs
import matplotlib.pyplot as plt

In [3]:
# Code inspired by the Pratical Lesson nº 6
def run_benchmark(benchmark="OpenML-CC18"):
    results = pd.DataFrame(columns=["dataset", "score"]) # create dataframe for results
    benchmark_suite = openml.study.get_suite(benchmark) # obtain the benchmark suite

    selected_task_ids = [31, 10101, 3913, 3, 3917, 53, 11, 16, 32, 3549, 12] # id of selected tasks

    for task_id in selected_task_ids:
        task = openml.tasks.get_task(task_id) # download the OpenML task
        features, targets = task.get_X_and_y()  # get the data
        
        # knn = KNeighborsClassifier() -> Simplified way to use KNN 
        KNN = make_pipeline(SimpleImputer(strategy='constant'),StandardScaler(),KNeighborsClassifier())
        
        score = np.mean(cross_val_score(KNN, features, targets, cv=10, scoring="roc_auc_ovr")) # calculate the metric for kfold
        results = pd.concat([results, pd.DataFrame([[task_id, score]], columns=results.columns)], ignore_index=True)
        
    results.to_csv("results.csv", index=False)

In [4]:
run_benchmark()

/tmp/ipykernel_1249/3740506616.py:1: FutureWarning: Starting from Version 0.15.0 `download_splits` will default to ``False`` instead of ``True`` and be independent from `download_data`. To disable this message until version 0.15 explicitly set `download_splits` to a bool.
  run_benchmark()
/home/codebind/.pyenv/versions/3.9.5/lib/python3.9/site-packages/openml/tasks/functions.py:442: FutureWarning: Starting from Version 0.15 `download_data`, `download_qualities`, and `download_features_meta_data` will all be ``False`` instead of ``True`` by default to enable lazy loading. To disable this message until version 0.15 explicitly set `download_data`, `download_qualities`, and `download_features_meta_data` to a bool while calling `get_dataset`.
  dataset = get_dataset(task.dataset_id, *dataset_args, **get_dataset_kwargs)
/tmp/ipykernel_1249/3423076950.py:10: FutureWarning: Support for `dataset_format='array'` will be removed in 0.15,start using `dataset_format='dataframe' to ensure your code

In [10]:
results = pd.read_csv("results.csv")
avg_rank = results['score'].mean()
avg_rank

0.8368680877800606

In [11]:
results

,dataset,score
0,31,0.746524
1,10101,0.563995
2,3913,0.754924
3,3,0.855901
4,3917,0.632683
5,53,0.900977
6,11,0.766746
7,16,0.993486
8,32,0.998598
9,3549,0.998642
